# CNN tutorial

This tutorial will introduce a simple Convolutional Neural Network (CNN) architecture to classify images.

As an example we will use the UC Merced dataset ( http://weegee.vision.ucmerced.edu/datasets/landuse.html ).
It consists of 21 categories, with 100 images each. The RGB images will be resized to `256 x 256` pixels.

## Tasks

1. Go through the code and run cell by cell (using the "Run" button or "shift+enter"). 
2. Complete the CNN architecture
3. Answer the questions

**Windows users:** Adjust the relative paths for your system.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import itertools

from sklearn.metrics import confusion_matrix, classification_report

import keras
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, GlobalAveragePooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, add, Reshape, Activation
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam, SGD
from keras.utils import np_utils, plot_model
from keras import backend as K
from keras import initializers
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

import os
from shutil import copyfile

# optional to load the original image files instead of numpy files
from PIL import Image



# set random seed
seed = 4321

## Load the dataset

Set the (relative) `data_path` on your system.


In [ ]:
# data source: http://weegee.vision.ucmerced.edu/datasets/landuse.html

category_names = [
 'agricultural',
 'airplane',
 'baseballdiamond',
 'beach',
 'buildings',
 'chaparral',
 'denseresidential',
 'forest',
 'freeway',
 'golfcourse',
 'harbor',
 'intersection',
 'mediumresidential',
 'mobilehomepark',
 'overpass',
 'parkinglot',
 'river',
 'runway',
 'sparseresidential',
 'storagetanks',
 'tenniscourt']


# load all data
N_images = 100  # per category
nb_classes = 21
patch_size = 256  # width and height in pixel
channels = 3

'''
# Load the original dataset
data_dir = 'data/UCMerced_LandUse/Images'
images_orig = []
labels_num = []
for i in range(len(category_names)):
    
    # load all images per class
    for j in range(N_images):        
        img_path = os.path.join(data_dir, category_names[i], category_names[i]+'{:02d}.tif'.format(j))
        
        img = Image.open(img_path)
        
        img = img.resize((patch_size, patch_size), Image.ANTIALIAS)
        arr = np.asarray(img)
        images_orig.append(arr)
        labels_num.append([i])

images_orig = np.array(images_orig)
labels_num = np.array(labels_num)

# Save the numpy datasets
np.save('data/UCMerced_images_orig.npy', images_orig)
np.save('data/UCMerced_labels_num.npy', labels_num)
'''

# Load the numpy datasets
images_orig = np.load('data/UCMerced_images_orig.npy')
labels_num = np.load('data/UCMerced_labels_num.npy')

# Convert labels to categorical 'one hot encoded vector' --> required format for softmax cross-entropy loss.
# Create an array of zeros with length nb_classes and put a 1 at the index of the true label.
labels = np_utils.to_categorical(labels_num, nb_classes)
print('example of a label with an integer coding: \n{}'.format(labels_num[333]))
print('the same example as a one-hot encoded: vector: \n{}'.format(labels[333]))

print(images_orig.shape, images_orig.dtype)
print(labels.shape, labels.dtype)

## Organize the raw data in new directories train, val, test

For training we want to load the data from individual directories.

Instead of running this you can download the splitted data directories from the Google drive.

In [ ]:
copy_data = False


# Collect all image paths
data_dir = 'data/UCMerced_LandUse/Images'
image_paths = []
for i in range(len(category_names)):

    # load all images per class
    for j in range(N_images):        
        img_path = os.path.join(data_dir, category_names[i], category_names[i]+'{:02d}.tif'.format(j))

        image_paths.append(img_path)

# Split data into training and test
image_paths = np.array(image_paths)

nb_images = image_paths.shape[0]
shuffled_indices = np.arange(nb_images)
np.random.seed(seed)
np.random.shuffle(shuffled_indices)

training_indices = shuffled_indices[:int(0.7*nb_images)]               # 70% for training
val_indices = shuffled_indices[int(0.7*nb_images):int(0.8*nb_images)]  # 10% for val
test_indices = shuffled_indices[int(0.8*nb_images):]                   # 20% for test

train_paths = image_paths[training_indices]
val_paths = image_paths[val_indices]
test_paths = image_paths[test_indices]

# Copy images to train, val, test directories

# make directories:
for split in ['train', 'val', 'test']:
    if not os.path.exists('data/UCMerced_LandUse/'+split):
        os.makedirs('data/UCMerced_LandUse/'+split)

def copy_images_to_split_directory(paths, split):
    for p in paths:
        dst=p.replace('UCMerced_LandUse/Images', 'UCMerced_LandUse/'+split)
        if not os.path.exists(os.path.dirname(dst)):
            os.makedirs(os.path.dirname(dst))
        copyfile(src=p, dst=dst)

if copy_data:
    copy_images_to_split_directory(train_paths, split='train')
    copy_images_to_split_directory(val_paths, split='val')
    copy_images_to_split_directory(test_paths, split='test')


## Visualize the images

**Task**:
* Which categories might be easy to classify for a CNN? Note three and explain why.
* Which category-pairs might be confused? Note three and explain why.

In [ ]:
# VISUALIZE IMAGES
def plotImages( images, n_images=5):
    fig, axes = plt.subplots(1, n_images, figsize=(10, 10))
    axes = axes.flatten()
    for img, ax in zip(images, axes):
        ax.imshow(img)
        ax.set_xticks(())
        ax.set_yticks(())
    plt.tight_layout()
    plt.show()

# Plot some examples for each category    
def plot_examples_per_category():
    for i in range(nb_classes):
        print(category_names[i])
        images_to_print = images_orig[i*N_images:i*N_images+N_images]
        np.random.seed(seed)
        np.random.shuffle(images_to_print)
        print(images_to_print.shape)
        plotImages(images_to_print)

plot_examples_per_category()

## CNN Architecture

**Task**: Implement the following convolutional neural network (CNN). Consider the information in the Figure as well as the additional remarks:

* Use `strides = (1, 1)` for all convolutional layers. 
* Use `padding='same'` to pad the inputs and preserve the spatial dimensions after the convolution.
* Use 'relu' as activation for both conv and dense layers, except the last layer that should output 'softmax' activations (pseudo class probabilities).
* The spatial extent is reduced by using max pooling with `pool_size = (3, 3)`. Set the parameter `strides`, such that the spatial extent is reduced by factor 2 in both spatial dimensions.
* Add a dropout layer with `prob_drop_hidden = 0.5` after the first dense layer.

![Figure simple CNN](figures/simple_CNN.png)
*Figure 1: Visualization of a simple CNN architecture*


**Questions**:
1. What is the shape of a single filter kernel in the second convolutional layer?
2. What is the shape of the output feature maps after the third convolutional layer (before the MaxPooling)?
3. `model.summary()` shows the dimension of each layer output. What is the first dimension (None) of our tensors corresponding to?

In [ ]:
# parameters:
input_shape = (patch_size, patch_size, channels)
pool_size = (3, 3)                  # size of pooling area for max pooling
prob_drop_hidden = 0.5              # drop probability for dropout @ fc layer

def simple_CNN():

    model = Sequential()

    # conv1 layer
    # NOTE: the input shape is only needed for the first layer
    model.add(Conv2D(filters=64, kernel_size=(3, 3), strides=(1,1), padding='same', activation='relu', 
                   input_shape=input_shape))   
    
    model.add(MaxPooling2D(pool_size=pool_size, strides=(TODO,TODO), padding='same'))
    

    # TODO ...
    
    
    # reduce the spatial dimensions to 1x1 by averaging the activations for each depth
    model.add(GlobalAveragePooling2D())
    
    # fc1 layer
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(rate=prob_drop_hidden))

    # The output of the model (softmax)
    
    # TODO ...

    return model


# clear the session first, such that layer names start with index 1
K.clear_session()

model_simple_CNN = simple_CNN()
model_simple_CNN.summary()


In [ ]:
# We could also plot the model graph
# Note: You might need further installations

# plot_model(model_simple_CNN, show_shapes=True, to_file='simple_CNN.png')


## Some helper functions

In [ ]:
# A custom Data generator to iterate over the training data and load the samples as batches
# (We will not use it today...)
def generator(features, labels, batch_size, patch_size, channels, nb_classes):
 # Create empty arrays to contain batch of features and labels#
 batch_features = np.zeros((batch_size, patch_size, patch_size, channels))
 batch_labels = np.zeros((batch_size, nb_classes))

 while True:
    for i in range(batch_size):
         # choose random index in features
         index = np.random.choice(a=features.shape[0], size=1)

         #Optional: Add some data augmentation here

         batch_features[i] = features[index, :, :, :]
         batch_labels[i] = labels[index]
    yield batch_features, batch_labels
    
    
# Function to re-initialize the weights before starting a new experiment/training
def initialize_weights(model, layer_name=None):
    session = K.get_session()
    if layer_name is None:
        for layer in model.layers: 
            if hasattr(layer, 'kernel_initializer'):
                print('initialize weights of layer: {}'.format(layer.name))
                layer.kernel.initializer.run(session=session)
    else:
        layer = model.get_layer(name=layer_name)
        if hasattr(layer, 'kernel_initializer'):
            print('initialize weights of layer: {}'.format(layer.name))
            layer.kernel.initializer.run(session=session)
            
            
# Helper function to plot training curves
def plot_train_val_accuracy_and_loss(history, legend_suffix="", figsize=(8,6)):
    # list all data in history
    print(history.keys())

    # summarize history for accuracy
    plt.figure(figsize=figsize)
    plt.plot(history['acc'])
    plt.plot(history['val_acc'])
    plt.title('Accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train_'+legend_suffix, 'val_'+legend_suffix], loc='lower right')
    plt.show()

    # summarize history for loss
    plt.figure(figsize=figsize)
    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    plt.title('Loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train_'+legend_suffix, 'val_'+legend_suffix], loc='upper right')
    plt.show()            

## Preprocess the input images
We zero-center and normalize each input feature (RGB channel) separately.
Thus, we substract the training mean and divide by the training standard deviation for each RGB channel separately. 

You don't have to run this cell. The training mean and std are already provided.

In [ ]:
# PREPROCESSING: NORMALIZE THE INPUT DATA
# as calculating the Mean and STD over the entire training data takes some time, the values are provided.

run_preprocessing = False


def get_mean_and_std_per_channel(data_train):
   
    if not data_train.dtype == np.float64:
        print('CONVERTING: to float64 for preprocessing')
        data_train = np.asarray(data_train, dtype=np.float64)

    mean = np.nanmean(data_train, axis=(0, 1, 2))
    std = np.nanstd(data_train, axis=(0, 1, 2))

    print('mean: ', mean)
    print('mean.shape', mean.shape, 'mean.dtype: ', mean.dtype)

    print('std: ', std)
    print('std.shape', std.shape, 'std.dtype: ', std.dtype)
        
    data_norm = normalize_images_per_channel(data_train, mean, std)
    
    # Control mean and std after preprocessing
    print('Check mean and std after preprocessing. Means should be close to zero. Stds should be close to one.')
    mean_control = np.nanmean(data_norm, axis=(0, 1, 2))
    std_control = np.nanstd(data_norm, axis=(0, 1, 2))
    print('mean_control', mean_control)
    print('std_control: ', std_control)

    return np.asarray(mean, dtype=np.float32), np.asarray(std, dtype=np.float32)


def normalize_images_per_channel(images, mean_train, std_train):
    normalized_images = images - mean_train
    normalized_images = normalized_images / std_train
    print('normalized_images.dtype', normalized_images.dtype)

    return normalized_images


def denormalize_images_per_channel(images, mean_train, std_train):
    denormalized_images = images * std_train
    denormalized_images = denormalized_images + mean_train
    print('denormalized_images.dtype', denormalized_images.dtype)

    return denormalized_images




if run_preprocessing:
    print('calculating mean and std per channel from the training data...')
    MEAN_train, STD_train = get_mean_and_std_per_channel(data_train=X_train)
    print(MEAN_train)
    print(STD_train)
    # Save mean and std to file
    # mean_train_file = data_dir + '/UCmerced_mean_train.npy'
    # std_train_file = data_dir + '/UCmerced_std_train.npy'
    # np.save(mean_train_file, MEAN_train)
    # np.save(std_train_file, STD_train)

    # Load mean and std from file
    # MEAN_train = np.load(mean_train_file)
    # STD_train = np.load(std_train_file)

    X_train_prepro = normalize_images_per_channel(images=X_train, mean_train=MEAN_train, std_train=STD_train)
    X_test_prepro = normalize_images_per_channel(images=X_test, mean_train=MEAN_train, std_train=STD_train)
    X_val_prepro = normalize_images_per_channel(images=X_val, mean_train=MEAN_train, std_train=STD_train)

    # Save preprocessed data split
    # np.savez('pretrained_model_simpleCNN/intermediate_results/dataset_TrainValTest_float32.npz',
    #        X_train_prepro=X_train_prepro, y_train=y_train,
    #        X_val_prepro=X_val_prepro, y_val=y_val,
    #        X_test_prepro=X_test_prepro, y_test=y_test,
    #        MEAN_train=MEAN_train, STD_train=STD_train)
 
    
print('done')

# Preprocessing function which is used by the data generator

As the kreas data generator expects a preprocessing function with only 1 input argument, we have hardcoded the training mean and std of this particular dataset and split here.

In [ ]:
MEAN_train = np.array([123.92658,  125.36055,  115.218414], dtype=np.float32)
STD_train = np.array([55.47791355, 51.37877802, 49.92980842], dtype=np.float32)

def normalize_UCMerced(image):
    np.array([image])
    
    mean_train = np.array([123.92658,  125.36055,  115.218414], dtype=np.float32)
    std_train = np.array([55.47791355, 51.37877802, 49.92980842], dtype=np.float32)
    
    normalized_image = image - mean_train
    normalized_image = normalized_image / std_train
    return normalized_image

## Train the model

The model was trained on a GPU (Titan Xp 12GB) for 200 epochs within 20 minutes (6 seconds per epoch). This code loads the weights after the epoch 200. You will train one epoch more and plot the training/validation curves for the entire 201 epochs. 

If you have not implemented the model correctly the weights of the pre-trained model will not match the model architecture. 

* Does the model learn anything better than a random classifier?
* Is the model overfitting?
* After how many epochs should we stop training? Or should we train even longer?



In [ ]:
# Set hyper parameters
# Set nb_epoch to 201 if you want to train 1 more epoch on your machine. Set it to 200 for no further training 
nb_epoch = 201
base_learning_rate=0.0001
batch_size = 2

In [ ]:
# Calculate the number of batches per epoch. 
# One epoch is defined as an update pass through the full training data. 
# One iteration is defined as an update through one mini-batch.
batches_per_epoch = train_paths.shape[0]//batch_size
# the number of batches to see the full validation data:
validation_steps = val_paths.shape[0]//batch_size
test_steps = test_paths.shape[0]
print('number of images per batch: {}'.format(batch_size))
print('batches per epoch: {}'.format(batches_per_epoch))
print('validation steps: {}'.format(validation_steps))
print('test steps: {}'.format(test_steps))

### Data augmentation

Which data augmentation techniques are useful for our dataset?

**Task**: Implement some data augmentation operations. Look at the implemented keras tools (https://keras.io/preprocessing/image/) and choose the one that make sense for this problem.
    


In [ ]:
# Data Generator provided by keras to load the training data in batches
# Usage if all data is in memory: generator=image_gen.flow(X_train_prepro, Y_train, batch_size=batch_size),

image_gen = ImageDataGenerator(
    preprocessing_function=normalize_UCMerced
    # TODO: Add some arguments for data augmentation ...

)

# At Test/Validation time without data augmentation
image_gen_test = ImageDataGenerator(preprocessing_function=normalize_UCMerced)

### Create iterators that read the data in batches from the train, val, or test directory

This needs much less memory than loading the entire training dataset into memory.
Especially the images converted to float32 consume much more memory.

However, data loading can be the bottleneck when training on a GPU. We would have to make sure that the data reading is fast enough and does not slow down the training process. This is usually regulated with a queue and multiple workers that read the images in parallel.

In [ ]:
train_it = image_gen.flow_from_directory('data/UCMerced_LandUse/train/', 
                                       classes=category_names, class_mode='categorical', 
                                       batch_size=batch_size, target_size=(256, 256))

val_it = image_gen_test.flow_from_directory('data/UCMerced_LandUse/val/', 
                                       classes=category_names, class_mode='categorical', 
                                       batch_size=batch_size, target_size=(256, 256))

# Let's look at the first return from the training iterator
batchX, batchy = train_it.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))


### Optimize

**Attention**: If your model architecture is not as expected, there will be an error when loading the weights, because the number of parameters is not matching.

In [ ]:
load_pre_trained_model = True  # load the pretrained model weights
continue_training = True if nb_epoch > 200 else False  # to load the history (loss curves) for the 200 pretrained epochs

# set your architecture
model = model_simple_CNN

# Define an optimizer
opt = Adam(lr=base_learning_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

if load_pre_trained_model:
    print('Loading weights from pre-trained model...')
    weights_path='pretrained_model_simpleCNN_v2/weights_simpleCNN_epoch200.h5'
    model.load_weights(filepath=weights_path, by_name=False, skip_mismatch=False)
    
    # Note: We could also load the full model (architecture + weights)
    #  model = load_model('pretrained_model_simpleCNN_v2/model_simpleCNN_epoch3.h5')
    initial_epoch = 200
else:
    # re-initialize weights of the model to train from scratch
    print('initializing weights...')
    initialize_weights(model)
    initial_epoch = 0

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

# Save model checkpoints/parameters if the performance has improved.
checkpoint_best_path = 'weights_best.hdf5'
checkpoint_best = ModelCheckpoint(checkpoint_best_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only=True, period=1)
        
# Fit the model parameters
print('training...')
history = model.fit_generator(
                generator=train_it,
                steps_per_epoch=batches_per_epoch, epochs=nb_epoch, 
                validation_data=val_it,
                validation_steps=validation_steps,
                callbacks=[checkpoint_best],
                initial_epoch=initial_epoch,
                workers = 2,
                max_q_size=10)

print('done')

### Plot the training history

In [ ]:
with open('pretrained_model_simpleCNN_v2/trainHistoryDict_epoch200.pkl', 'rb') as f:
    history_pretrained = pickle.load(f, encoding='latin1')

        
if continue_training:
    # Load the previous history
    # Merge previous history until epoch 200 with new history
    for key in history_pretrained.keys():
            history.history[key] = history_pretrained[key] + history.history[key]    
else:
    history.history = history_pretrained
    # # Save the history
    with open('trainHistoryDict_epoch{}.pkl'.format(nb_epoch), 'wb') as file:
        pickle.dump(history.history, file)
        
# Save the model state of the last epoch    
model.save_weights('weights_simpleCNN_epoch{}.h5'.format(nb_epoch))
model.save('model_simpleCNN_epoch{}.h5'.format(nb_epoch))
print('done')

# Plot history
plot_train_val_accuracy_and_loss(history.history, legend_suffix='original', figsize=(8,6))


##  Prediction on the test data

In [ ]:
# create another iterator that load all test images and labels with one iteration (needed for later analysis)
test_it_load = image_gen_test.flow_from_directory('data/UCMerced_LandUse/test/', 
                                       classes=category_names, class_mode='categorical', 
                                       batch_size=test_steps, target_size=(256, 256),
                                       shuffle=False)
X_test_prepro, y_test = test_it_load.next()
print('test steps:', test_steps)
print(X_test_prepro.shape)

# convert the test labels from one hot encoded to integers
y_test_num = np.argmax(np.squeeze(y_test), axis=1)
y_test_cat = [category_names[l] for l in y_test_num]
print('num. test labels: ', len(y_test_cat))

In [ ]:
run_prediction = True  # set this to false to load the saved predictions

if run_prediction:
    # load the weights form the best epoch
    model.load_weights(filepath='pretrained_model_simpleCNN_v2/weights_best.hdf5', by_name=False, skip_mismatch=False)
    print('predicting...')
    predictions = model.predict(x=X_test_prepro, batch_size=batch_size, steps=None)
    np.save('predictions.npy', predictions)
else:
    print('load predictions...')
    predictions = np.load('pretrained_model_simpleCNN_v2/predictions.npy')

# convert predictions    
predictions_num = np.argmax(predictions, axis=1)
predictions_cat = [category_names[pred] for pred in predictions_num]
print('num. predictions: ', len(predictions_cat))



In [ ]:
# To visualize some random samples shuffle the dataset 
np.random.seed(seed)
np.random.shuffle(X_test_prepro)
np.random.seed(seed)
np.random.shuffle(predictions_cat)
np.random.seed(seed)
np.random.shuffle(y_test_cat)
np.random.seed(seed)
np.random.shuffle(predictions_num)
np.random.seed(seed)
np.random.shuffle(y_test_num)

## Evaluation on the test data

1. Which categories are mainly confused?
2. Which categories are easy for the classifier?
3. Compare the results to the assessment you did at the beginning! How was your intuition?

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.gray,
                          filepath=None,
                          figsize=(12,12)):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    fig = plt.figure(figsize=figsize)

    plt.imshow(cm, interpolation='nearest', cmap=cmap, vmin=0, vmax=1.0)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90, fontsize=16)
    plt.yticks(tick_marks, classes, fontsize=16)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black", fontsize=9)

    plt.ylabel('True label', fontsize=22)
    plt.xlabel('Predicted label', fontsize=22)
    plt.tight_layout()

    if filepath is not None:
        fig.savefig(filepath)
                

# classification report
report_string = classification_report(y_test_num, predictions_num, target_names=category_names)
print(report_string)
                
# Overall accuracy
print('num. correct samples:  {} \nnum. all test samples: {}'.format(np.sum(y_test_num == predictions_num) , predictions_num.shape[0]))
overall_acc = np.sum(y_test_num == predictions_num) / float(predictions_num.shape[0])
print('overall accuracy: {:.2f}'.format(overall_acc))

# Confusion matrix
confmat = confusion_matrix(y_test_num, predictions_num )
plot_confusion_matrix(cm=confmat, classes=category_names, normalize=True, cmap=plt.cm.Blues, filepath=None)


## Visualization of some test examples

**Task**: Let's look at some specific error cases (highlighted with a red frame). Can we explain why the network might fail in some of the cases?

In [ ]:
# VISUALIZE IMAGES
def plotImages_categories( images, labels, predictions, n_rows=5, n_cols=4, figsize=(10, 10)):
    fig, axes = plt.subplots(n_rows, n_cols, figsize=figsize)
    axes = axes.flatten()
    for i in range(len(axes)):
        
        axes[i].imshow(images[i])
        axes[i].set_xticks(())
        axes[i].set_yticks(())
        if predictions is not None:
            title = 'GT:   {}\nPRED: {}'.format(labels[i], predictions[i])
            
            if predictions[i] != labels[i]:
                [j.set_linewidth(5) for j in axes[i].spines.values()]
                [j.set_color('red') for j in axes[i].spines.values()]

        else:
            title = 'GT: {}'.format(labels[i])
        axes[i].set_title(title, fontdict={'family':'monospace'}, loc='left')
    plt.tight_layout()
    plt.show()

# prepare the images for visualization: 1. denormalize, 2. convert to unint8
X_test_vis = np.asarray(denormalize_images_per_channel(images=X_test_prepro, mean_train=MEAN_train, std_train=STD_train),
                        dtype=np.uint8)

plotImages_categories(images=X_test_vis, labels=y_test_cat, predictions=predictions_cat, 
                      n_rows=20, n_cols=4, figsize=(12, 60))

## Bonus ... or homework

1. What is the effect of preprocessing/normalizing the input data? Train and test without preprocessing, but still convert the images from uint8 to float32.
2. Similar, test the effect of the data augmentation. Run a baseline without data augmentation.
3. Try to improve the performance by modifying/extending the CNN architectrue.

Our implemented CNN is a pretty simple, yet powerful enough to learn generalizing features that solve this scene classification task with a decent performance.

Below, is the code for the famous ResNet50 architecture [1].
It is a very deep network constructed with building blocks. Within each block a skip connection directly forwards the input to the output. These skip connections allow to train much deeper networks and help to propagate the gradient during the optimization procedure.

As a homework you may want to look at the code and the model summary (i.e. look at the number of trainable parameters). 
Furthermore, you could try to train the ResNet50 on the UC Merced dataset from scratch or fine-tune the weights trained on ImageNet.

**Note**: Once you start using CNNs for your own projects and datasets, it might be a good idea to take a state-of-the-art architecture such as the ResNet50 and apply it to your new problem. Usually we can also find some pre-trained weights that can be used to initialize the network. This may be crucial, especially when your reference dataset is too small to train such a deep network from scratch.

After testing the state-of-the-art you may want to modify the existing architecture to better suite your specific task. Think about what is specific about your data and the task you want to solve.

**Keras already provides many CNN architectures including pre-trained weights**

For an easy start, check out: https://keras.io/applications/

[1] He, Kaiming, et al. "Deep residual learning for image recognition." Proceedings of the IEEE conference on computer vision and pattern recognition. 2016.

In [ ]:
def identity_block(input_tensor, kernel_size, filters, stage, block):
    """
    The identity_block is the block that has no conv layer at shortcut
    Arguments
        input_tensor: input tensor
        kernel_size: defualt 3, the kernel size of middle conv layer at main path
        filters: list of integers, the nb_filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    """

    nb_filter1, nb_filter2, nb_filter3 = filters
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters=nb_filter1, kernel_size=(1, 1), name=conv_name_base + '2a')(input_tensor)

    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters=nb_filter2, kernel_size=(kernel_size, kernel_size), 
               padding='same', name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters=nb_filter3, kernel_size=(1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    x = add([x, input_tensor])
    x = Activation('relu')(x)
    return x


def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):
    """
    conv_block is the block that has a conv layer at shortcut
    # Arguments
        input_tensor: input tensor
        kernel_size: defualt 3, the kernel size of middle conv layer at main path
        filters: list of integers, the nb_filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    Note that from stage 3, the first conv layer at main path is with subsample=(2,2)
    And the shortcut should have subsample=(2,2) as well
    """

    nb_filter1, nb_filter2, nb_filter3 = filters
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters=nb_filter1, kernel_size=(1, 1), strides=strides,
                      name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters=nb_filter2, kernel_size= (kernel_size, kernel_size), padding='same',
                      name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters=nb_filter3, kernel_size=(1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    shortcut = Conv2D(filters=nb_filter3, kernel_size=(1, 1), strides=strides,
                             name=conv_name_base + '1')(input_tensor)
    shortcut = BatchNormalization(axis=bn_axis, name=bn_name_base + '1')(shortcut)

    x = add([x, shortcut])
    x = Activation('relu')(x)
    return x



def ResNet50(img_input):
    model = Sequential()

    x = Conv2D(64, kernel_size=(7, 7), strides=(2, 2), name='conv1', padding='same')(img_input)
    x = BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)


    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')
    
    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a', strides=(1, 1))
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')
    
    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a', strides=(1, 1))
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f')

    x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a', strides=(1, 1))
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')

    # Fully Connected Softmax Layer

    x_fc = GlobalAveragePooling2D()(x)
    x_fc = Dense(nb_classes, activation='softmax', name='your_output')(x_fc)

    model = Model(img_input, x_fc)
    return model


input_shape = (patch_size, patch_size, channels)
img_input = Input(shape=input_shape)
bn_axis = 3

model_resnet50 = ResNet50(img_input)
model_resnet50.summary()